# Fitting a PoSH potential

### Loading the required modules

In [16]:
using IPFitting, SHIPs, JuLIP, LinearAlgebra, HDF5

### Reading in the configurations (Si)

In [2]:
cfgs = IPFitting.Data.read_xyz("Si.xyz")[1:50:end];

Reading in Si.xyz ...
Processing data ...


Progress: 100%|█████████████████████████████████████████| Time: 0:00:06


### Configuration types in the training database

In [3]:
@show unique(configtype.(cfgs));

unique(configtype.(cfgs)) = ["isolated_atom", "sh", "dia", "bt", "liq", "amorph", "surface_001", "surface_111", "surface_111_pandey", "crack_110_1-10", "sp", "sp2", "interstitial", "divacancy", "vacancy", "decohesion", "bc8", "bcc", "fcc", "hcp", "hex_diamond", "st12"]


### Filtering out a specific configuration type (isolated_atom)

In [4]:
cfgs = cfgs[ findall(configtype.(cfgs) .!= "isolated_atom") ];

49-element Array{Dat,1}:
 Dat(Atoms{Float64}(StaticArrays.SArray{Tuple{3},Float64,1,3}[[0.0, 0.0, 0.0]], StaticArrays.SArray{Tuple{3},Float64,1,3}[[0.0, 0.0, 0.0]], [28.085], Int16[14], [2.41498 0.0 0.0; -1.49314 2.43193 0.0; -0.08399 0.391257 2.39604], Bool[true, true, true], nothing, JuLIP.DofManager{Float64}(false, [1, 2, 3], JuLIP.LinearConstraint{Float64}[], StaticArrays.SArray{Tuple{3},Float64,1,3}[], [0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0]), Dict{Any,JuLIP.JData{Float64}}()), "sh", Dict("V"=>[1.52353, 0.357122, 1.18801, -0.452278, 0.173689, 0.197629],"E"=>[-162.692],"F"=>[0.0, 0.0, 0.0]), Dict{String,Array{Int64,1}}(), Dict{String,Any}())                                                                                                                                                                                                                                                                                                                                                           

## Create the PoSH Basis

- Set the inner/outer cutoff (rcut_in, rcut_out)
- Set the cutoff function and space transform

Here `N` is the number of interaction neighbours, `polydeg` the maximum degree of polynomial fit, `wL` the weight on the angular part (lower increases accuracy at the expense of cost) and `chc` the sparsification.

In [10]:
r0 = rnn(:Si)
rcut_in = 0.68*r0
rcut_out = 5.2       

trans = SHIPs.PolyTransform(2, r0)
cutf = SHIPs.PolyCutoff1s(2, 0.0, rcut_out)

N = 4
polydeg = 12
wL = 1.4
chc = 0.1

shipbas### Filtering out a specific configuration type (isolated_atom)is = SHIPBasis(SparseSHIP(:Si, N, polydeg; wL = wL, chc = chc), trans, cutf);

### Numer of basis functions

In [20]:
length(shipbasis)

761

## Assemble the LsqDB

This will assemble the least squares matrix and save as a database with two files `_info.json` and `_kron.h5` which contain the data

In [12]:
dB = LsqDB("Si", shipbasis, cfgs);

Assemble LSQ blocks in serial


Progress: 100%|█████████████████████████████████████████| Time: 0:00:59m
┌ Info: Elapsed: 59.0s
└ @ IPFitting.Tools /Users/Cas/.julia/packages/IPFitting/4xRUd/src/tools.jl:68
┌ Info: Writing db to disk...
└ @ IPFitting.DB /Users/Cas/.julia/packages/IPFitting/4xRUd/src/lsq_db.jl:194
┌ Warning: The file Si_info.json already exists. It will be renamed to Si_info.json.psovn to avoid overwriting.
└ @ IPFitting.DB /Users/Cas/.julia/packages/IPFitting/4xRUd/src/lsq_db.jl:82
┌ Warning: The file Si_kron.h5 already exists. It will be renamed to Si_kron.h5.izfft to avoid overwriting.
└ @ IPFitting.DB /Users/Cas/.julia/packages/IPFitting/4xRUd/src/lsq_db.jl:82
┌ Info: ... done
└ @ IPFitting.DB /Users/Cas/.julia/packages/IPFitting/4xRUd/src/lsq_db.jl:201


## Fitting the PoSH

Fitting the PoSH potential requires the basis `shipbasis` and the least squares database `dB`. `lsqfit` will perform this but requires us to specify the isolated atom energy `E0` and the weights per configuration type. 

The `default` weight setting is the default weight for a configuration type, specifying the weights for a specific basis will overwrite this default setting.

In [14]:
E0 = -158.54496821

weights = Dict(
        "default" => Dict("E" => 100.0, "F" => 1.0 , "V" => 2.0 ),
            "liq" => Dict("E" =>  33.0, "F" => 0.66, "V" => 0.5 ),
         "amorph" => Dict("E" =>  10.0, "F" => 0.5 , "V" => 0.25),
             "sp" => Dict("E" =>  10.0, "F" => 0.5 , "V" => 0.25)
   )

Dict{String,Dict{String,Float64}} with 4 entries:
  "amorph"  => Dict("V"=>0.25,"E"=>10.0,"F"=>0.5)
  "default" => Dict("V"=>2.0,"E"=>100.0,"F"=>1.0)
  "liq"     => Dict("V"=>0.5,"E"=>33.0,"F"=>0.66)
  "sp"      => Dict("V"=>0.25,"E"=>10.0,"F"=>0.5)

### Saving the QR decomposition for regularisation

Performing the fit using a QR fit. The targets `y`, upper triangular matrix `R` and orthogonal matrix `Q` are saved in `_qr.h5`. This is only required for regularisation in the next notebook. 

In [17]:
saveqr = Dict()
GC.gc()
shipIP, lsqinfo = lsqfit(  dB; E0 = E0,
                                weights = weights,
                                 solver = (:qr, ),
                                 #deldb = true,    # this is to save some memory
                                 #asmerrs = false, # but we then cannot asm errors
                                 saveqr = saveqr)



lsqsys = nothing
IPFitting.Lsq._show_free_mem()

R = saveqr["R"]
Nb = size(R, 1)
Y = saveqr["Y"]
@time y = (Y' * saveqr["Q"])[1:Nb]
_rmse = norm(Y - saveqr["Q"] * y)
h5open("Si_qr.h5","w") do h5
    h5["R"] = R
    h5["y"] = y
    h5["rmse"] = _rmse
end

┌ Info: assemble lsq system
└ @ IPFitting.Lsq /Users/Cas/.julia/packages/IPFitting/4xRUd/src/lsq.jl:305
┌ Info: Free Memory: ≈ 0.04 GB
└ @ IPFitting.Lsq /Users/Cas/.julia/packages/IPFitting/4xRUd/src/lsq.jl:249
┌ Info: Free Memory: ≈ 0.04 GB
└ @ IPFitting.Lsq /Users/Cas/.julia/packages/IPFitting/4xRUd/src/lsq.jl:249
┌ Info: Free Memory: ≈ 0.04 GB
└ @ IPFitting.Lsq /Users/Cas/.julia/packages/IPFitting/4xRUd/src/lsq.jl:249
┌ Info: solve (10480, 761) LSQ system using QR factorisation
└ @ IPFitting.Lsq /Users/Cas/.julia/packages/IPFitting/4xRUd/src/lsq.jl:322
┌ Info: cond(R) = 6.657139110726477e36
└ @ IPFitting.Lsq /Users/Cas/.julia/packages/IPFitting/4xRUd/src/lsq.jl:326


    Status `~/.julia/environments/v1.1/Project.toml`
  [3002bd4c] IPFitting v0.3.2


┌ Info: Free Memory: ≈ 0.03 GB
└ @ IPFitting.Lsq /Users/Cas/.julia/packages/IPFitting/4xRUd/src/lsq.jl:249
┌ Info: Relative RMSE on training set: 0.0019349670763230576
└ @ IPFitting.Lsq /Users/Cas/.julia/packages/IPFitting/4xRUd/src/lsq.jl:362
┌ Info: Assemble Information about the fit
└ @ IPFitting.Lsq /Users/Cas/.julia/packages/IPFitting/4xRUd/src/lsq.jl:406


  0.009783 seconds (10 allocations: 94.406 KiB)


┌ Info: Free Memory: ≈ 0.03 GB
└ @ IPFitting.Lsq /Users/Cas/.julia/packages/IPFitting/4xRUd/src/lsq.jl:249


8.2091789840716

### Calculating the errors

We can add the errors using the `add_fits!` command. Using the `rmse_table` the errors are printed in table

In [18]:
add_fits!(shipIP, cfgs; fitkey="fit") #use add_fits! also crashse!!
rmse_test, rmserel_test = rmse(cfgs; fitkey="fit")
lsqinfo["errors"] = Dict("rmse" => rmse_test, "relrmse" => rmserel_test)

rmse_table(lsqinfo["errors"])

Add Fit info to configs in serial


Progress: 100%|█████████████████████████████████████████| Time: 0:00:02
┌ Info: Elapsed: 2.2s
└ @ IPFitting.Tools /Users/Cas/.julia/packages/IPFitting/4xRUd/src/tools.jl:68


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ RMSE                                                               ┃
┣━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━┫
┃ config type  ┃     E [meV]     │     F [eV/A]    │     V [meV]     ┃
┠──────────────╂────────┬────────┼────────┬────────┼────────┬────────┨
┃           sh ┃  10.77 ┊ 0.007% │  0.062 ┊ 18.17% │  160.7 ┊ 17.61% ┃
┃      vacancy ┃   5.13 ┊ 0.003% │  0.068 ┊  8.87% │   25.5 ┊ 14.72% ┃
┃          sp2 ┃   1.96 ┊ 0.001% │  0.104 ┊ 16.18% │    NaN ┊   NaN% ┃
┃          fcc ┃  20.15 ┊ 0.012% │  0.081 ┊ 39.09% │   51.3 ┊ 25.36% ┃
┃           bt ┃  21.04 ┊ 0.013% │  0.042 ┊ 15.51% │   60.2 ┊  4.52% ┃
┃ crack.._1-10 ┃  21.47 ┊ 0.013% │  0.123 ┊ 21.47% │   34.8 ┊ 11.52% ┃
┃          dia ┃  10.18 ┊ 0.006% │  0.037 ┊ 10.24% │   25.5 ┊  1.93% ┃
┃    divacancy ┃   3.65 ┊ 0.002% │  0.090 ┊  9.10% │    NaN ┊   NaN% ┃
┃ interstitial ┃   6.49 ┊ 0.004% │  0.077 ┊  8.47% │    8.3 ┊  9.30% ┃
┃     

Saving the unregularised PoSH potential

In [19]:
save_json("PoSH_Si_fit_unreg.json", Dict("IP" => Dict(shipIP), "info" => lsqinfo))